In [1]:
import sys
sys.path.append('/home/eric/scraping_houses/flaskApp')
sys.path.append('/home/eric/scraping_houses')

import pandas as pd
import dash_table
import json
from dash.dependencies import Input, Output
from sqlalchemy import text
from app import db
from app.models import Listing, Fence
import dash
import dash_core_components as dcc
import dash_html_components as html
from helpers.taxes import TaxBrackets, tax_calculator

# Getting data from SQLite
sql = text(f'''SELECT id,
                      centris_id,
                      category,
                      price,
                      geofence,
                      potential_revenue,
                      residential_units,
                      commercial_units,
                      unites_residentielles,
                      unite_principale,
                      centris_detail_url,
                      broker_detail_url
                  FROM Listings
                  ''')

df = pd.read_sql(sql, db.engine)
df['pt_revenue'] = df.potential_revenue/df.price
df = df.astype({'residential_units': float,
                'commercial_units': float})

df['welcome_tax'] = tax_calculator(df,
                        TaxBrackets.welcome_tax.get('Montreal'),
                        ).loc[:,'total']


df['school_tax'] = tax_calculator(df,
                        TaxBrackets.school_tax.get('Montreal'),
                        ).loc[:,'total']

df['property_tax'] = tax_calculator(df,
                        TaxBrackets.property_tax.get('Montreal'),
                        ).loc[:,'total']



In [2]:
def monthly_payments(price, interest_rate=3.79, years=20):
           
    # Interest rate monthly
    mrate=interest_rate/100/12
    nper=years*12
    monthly_payment = (price*(mrate*(1+mrate)**nper)/(((1+mrate)**nper)-1))
    
    return monthly_payment

In [3]:
renos= 10000
df['Total upfront'] = df.apply(lambda x: x.price+x.welcome_tax+renos, axis=1)
df['Monthly Payment'] = df.apply(lambda x: monthly_payments(x['Total upfront']), axis=1)
df['Yearly revenue'] = df.apply(lambda x: x.potential_revenue-(x.school_tax+x.property_tax), axis=1)
df['Monthly cashflow'] = df.apply(lambda x: x['Yearly revenue']/12 - x['Monthly Payment'], axis=1)

In [6]:
selected_row = df.loc[3,:]
selected_row.centris_id

28941265

In [7]:
selected_row.index

Index(['id', 'centris_id', 'category', 'price', 'geofence',
       'potential_revenue', 'residential_units', 'commercial_units',
       'unites_residentielles', 'unite_principale', 'centris_detail_url',
       'broker_detail_url', 'pt_revenue', 'welcome_tax', 'school_tax',
       'property_tax', 'Total upfront', 'Monthly Payment', 'Yearly revenue',
       'Monthly cashflow'],
      dtype='object')